# Analisa perusahaan startup buku

Dikarenakan adanya virus corona, membuat rutinitas membaca buku di kalangan masyarakat menjadi meningkat. Perusahaan startup yang bergerak dibidang buku ingin mengembangkan suatu aplikasi. Perusahaan tersebut ingin mengetahui beberapa informasi sehingga bisa digunakan untuk membuat suatu penawaran ketika terdapat produk baru. Informasi ini berasal dari database yang berisi buku, penerbit, penulis, rating, dan ulasan pelanggan. Sehingga dari database ini, akan diolah menjadi suatu kumpulan informasi yang berguna untuk perusahaan tersebut.

Langkah - langkah analisa adalah melakukan koneksi ke database, mempelajari tabel, melakukan queri dan memberikan kesimpulan

## Melakukan koneksi ke database

Pertama akan dilakukan koneksi ke database sehingga bisa dilakukan pengqueryan dari database

In [21]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'practicum_student',         # nama pengguna
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # kata sandi
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # port koneksi
             'db': 'data-analyst-final-project-db'}          # nama database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Mempelajari tabel

Selanjutnya akan diakses tabel untuk mempelajari isi dari tabel

In [22]:
books = '''SELECT * FROM books LIMIT 5;'''
authors = '''SELECT * FROM authors LIMIT 5;'''
ratings = '''SELECT * FROM ratings LIMIT 5;'''
reviews = '''SELECT * FROM reviews LIMIT 5;'''
publishers = '''SELECT * FROM publishers LIMIT 5;'''

In [23]:
tests = [books, authors, ratings, reviews, publishers]
for test in tests:
    print('nama database:', test[14:-8])
    query = pd.io.sql.read_sql(test, con = engine) 
    print(query)
    print('------------------------------------------')

nama database: books 
   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  
------------------------------------------
nama database: authors 
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christi

Dari database, terdapat 5 tabel yang memiliki hubungan khusus satu sama lain. Pada tabel`books`, terdapat 6 kolom dengan adanya primary key pada `book_id` dan foreign key pada `author_id` dan `publisher_id`. Untuk tabel `authors`, terdapat 2 kolom dengan primary key pada `author_id`. Pada tabel `ratings`, terdapat 4 kolom dengna primary key adalah `rating_id` dan foreign key nya adalah `book_id`. Pada kolom `reviews`, terdapat 2 kolom dengan primary key adalah `review_id` dan foreign key adalah `book_id`. Pada kolom `publishers`, terdapat 2 kolom dengan primary key adalah `publisher_id`.

## Queri SQL

Selanjutnya akan dilakukan pengquerian dari database sehingga bisa menghasilkan informasi yang menarik dari database

### Jumlah buku yang rilis setelah tahun 2000

In [24]:
task_1 = '''SELECT COUNT(book_id) AS Total_buku 
            FROM books 
            WHERE EXTRACT(YEAR FROM CAST(publication_date as date)) > 2000;
         '''
query = pd.io.sql.read_sql(task_1, con = engine) 
print(query)

   total_buku
0         783


Dari tabel `books`, diperoleh bahwa terdapat 783 buku yang terbit setelah tahun 2000.

### Jumlah ulasan dan rating rata - rata setiap buku

In [25]:
task_2 = '''SELECT books.title,
            subq.average_rating,
            subq.count_review
            FROM
                (SELECT reviews.book_id as book_id,
                   COUNT (reviews.review_id) AS count_review,
                   AVG (ratings.rating) AS average_rating
                   FROM reviews
                   LEFT JOIN ratings ON ratings.book_id = reviews.book_id
                   GROUP BY reviews.book_id
                   ORDER BY count_review DESC) AS subq
            LEFT JOIN books ON subq.book_id = books.book_id
            ORDER BY
            count_review DESC
            LIMIT 5;'''
query = pd.io.sql.read_sql(task_2, con = engine) 
print(query)

                                               title  average_rating  \
0                            Twilight (Twilight  #1)        3.662500   
1                The Hobbit  or There and Back Again        4.125000   
2                             The Catcher in the Rye        3.825581   
3  Harry Potter and the Prisoner of Azkaban (Harr...        4.414634   
4  Harry Potter and the Chamber of Secrets (Harry...        4.287500   

   count_review  
0          1120  
1           528  
2           516  
3           492  
4           480  


Dari tabel `books` dan penyatuan tabel `reviews` dan `ratings`, didapat bahwa jumlah ulasan terbanyak adalah buku Twilight pertama dengan jumlah review 1120 review dan rating rata - rata adalah 3,66, lalu diikuti oleh The Hobbit dengan total review 528 dan rata - rata rating 4,12. Dari 5 buku dengan jumlah review terbanyak, dapat dilihat bahwa buku dengan genre fiksi mendominasi jumlah review terbanyak.

### Penerbit yang menerbitkan jumlah buku terbanyak

In [26]:
task_3 = '''SELECT publishers.publisher AS publisher, 
            COUNT(books.book_id) AS total_book
            FROM books
            LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
            WHERE books.num_pages > 50
            GROUP BY publisher
            ORDER BY COUNT(books.book_id) DESC
            LIMIT 3;
         '''
query = pd.io.sql.read_sql(task_3, con = engine) 
print(query)

                  publisher  total_book
0             Penguin Books          42
1                   Vintage          31
2  Grand Central Publishing          25


Dari menggabungkan tabel `books` dan `publishers`, didapat bahwa penerbit yang menerbitkan buku terbanyak dengan jumlah halaman buku lebih dari 50 halaman adalah Penguin Books dengan total buku 42 buah, diikuti oleh Vintage dengan 31 buah, dan Grand Central Publishing dengan 25 buah.

### Penulis dengan rating rata - rata tertinggi

In [27]:
task_4 = '''SELECT authors.author AS author,
            AVG(subq_2.average_rating) AS filter_average
            FROM
                (SELECT books.title,
                books.author_id,
                subq_1.average_rating
                FROM
                    (SELECT book_id,
                     COUNT (rating_id) AS count_rating,
                     AVG (rating) AS average_rating
                     FROM ratings
                     GROUP BY book_id
                     HAVING COUNT (rating_id) > 50) AS subq_1
                LEFT JOIN books ON books.book_id = subq_1.book_id) AS subq_2
            LEFT JOIN authors ON authors.author_id = subq_2.author_id
            GROUP BY author
            ORDER BY filter_average DESC
              LIMIT 5;
              '''
query = pd.io.sql.read_sql(task_4, con = engine) 
print(query)

                              author  filter_average
0         J.K. Rowling/Mary GrandPré        4.283844
1  Markus Zusak/Cao Xuân Việt Khương        4.264151
2                     J.R.R. Tolkien        4.258446
3                  Louisa May Alcott        4.192308
4                       Rick Riordan        4.080645


Dari hasil querian untuk mencari penulis dengan rata - rata rating tertinggi dengan jumlah rating lebih dari 50, didapat bahwa  J.K. Rowling/Mary GrandPré memiliki rata - rata rating tertinggi dengan skor 4,28, lalu diikuti oleh 1  Markus Zusak/Cao Xuân Việt Khương dengan skor 4,26, dan J.R.R. Tolkien dengan skor 4.25. Dapat dilihat pula bahwa 5 penulis dengan rata - rata rating terbesar memiliki rating diatas 4 dari maksimal skor 5.

### Jumlah rata - rata teks ulasan 

In [29]:
task_5 = '''SELECT AVG(subq_2.count_review) AS average_count_review
            FROM (SELECT COUNT (reviews.review_id) AS count_review,
                  subq_1.username  
                  FROM (SELECT username,
                        COUNT(rating_id) AS count_rating
                        FROM ratings
                        GROUP BY username
                        HAVING COUNT (rating_id) > 50) AS subq_1
                  LEFT JOIN reviews ON reviews.username = subq_1.username
                  GROUP BY subq_1.username) AS subq_2;
         '''
query = pd.io.sql.read_sql(task_5, con = engine) 
print(query)

   average_count_review
0             24.333333


Dari hasil pengquerian, rata - rata pengguna yang memberikan rating untuk lebih dari 50 buku memberikan 24 teks ulasan.